In [21]:
from __future__ import print_function, division
import time, os
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas
import glob
from sklearn import preprocessings

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
snapshots_df_full = pandas.read_csv('/home/cs231n/data/Field_Snaps_With_warranty.txt')
selected_snapshots_full = snapshots_df_full[['Veh Ref ID',
 'Event DateTime',
 'Event Type Description',
 'Acc Pedal Position',
 'Ambient Air Temp',
 'Barometric Press',
 'Brake Switch',
 'Bus Utilization',
 'Cat Intake Gas Temp',
 'Cat Outlet Gas Temp',
 'Clutch Switch',
 'Cmd Eng Fuel Press',
 'Cruise Status',
 'Dpf Regen Inhibit Sw',
 'Dpf Thermal Mngmnt',
 'Drvr Demand Torque',
 'Eng Air Flow Rate',
 'Eng Avg Fuel Econ',
 'Eng Coolant Level',
 'Eng Coolant Temp',
 'Eng Demand Torque',
 'Eng DPF Intake Press',
 'Eng Egr Valve Pos',
 'Eng Exhaust Gas Temp',
 'Eng Fuel Del Press',
 'EngFuelTemp1',
 'Engine Speed',
 'Eng Man Abs Pressure',
 'Eng Oil Pressure',
 'EngInjRail1Press',
 'EngIntakeMan1Temp',
 'EngOilTemp1',
 'Eng Percent Torque',
 'EngTurbo1Boost',
 'EngTurbo1Pos',
 'EngTurbo1Speed',
 'Event - All Lamps On Time Hr',
 'Event - Amber Lamp Time Hr',
 'Event - Mil Lamp Time Hr',
 'Event - Red Lamp Time Hr',
 'Exhaust Tank Level',
 'Exhaust Tank Temp',
 'Fan Speed',
 'Keyswitch Bat Pot',
 'Part Trap Diff Press',
 'Part Trap Out Temp',
 'Scr Intake Gas Temp',
 'Scr Outlet Gas Temp',
 'Vehicle Speed',
 'DTCID',
 'Trip Distance',
 'Trip Idle Time',
 'Trip Run Time',
 'Altitude',
 'Engine Start Ambient',
 'Engine Start Coolant',
 'Latitude',
 'Longitude',
 'Lifetime Idle Hours',
 'Lifetime Idle Fuel',
 'Lifetime Fuel',
 'Lifetime Distance',
 'Lifetime Engine Hours']]

In [11]:
len(snapshots_df_full)

1581892

In [12]:
snapshots_full_deduped = snapshots_df_full.drop_duplicates()

In [13]:
len(snapshots_full_deduped)

1015071

In [14]:
selected_repairs_full = pandas.read_csv('/home/cs231n/data/repairs.csv')[[
 'Chassis\nReference\nNumber',
 'Model Vehicle',
 'Build_Dt',
 'Dlvry_Dt',
 'In Service Date',
 'Miles',
 'Rpr_Dt',
 'ATA3',
 'ATA3Desc',
 'ATA6',
 'ATA6Desc',
 'ATA9',
 'ATA9Desc',
 'Fail Type',
 'Repair Cost']]
selected_repairs_full = selected_repairs_full[selected_repairs_full['Chassis\nReference\nNumber'].notnull()] 

In [15]:
# don't include repairs before delivery
selected_repairs = selected_repairs_full[selected_repairs_full['Miles']>0]

In [5]:
#selected_repairs = selected_repairs_full[selected_repairs_full['Repair Cost'].isin(['medium', 'high ', 'very high'])]
veh_ids = selected_repairs['Chassis\nReference\nNumber'].unique()
selected_snapshots = selected_snapshots_full[selected_snapshots_full['Veh Ref ID'].isin(veh_ids)]

In [16]:
veh_ids = selected_repairs['Chassis\nReference\nNumber'].unique()
selected_snapshots_full_deduped = snapshots_full_deduped[snapshots_full_deduped['Veh Ref ID'].isin(veh_ids)]

In [17]:
len(selected_snapshots_full_deduped)

851333

In [18]:
selected_snapshots_full_deduped_backfilled = selected_snapshots_full_deduped.fillna(method='bfill')

In [19]:
selected_snapshots_full_deduped_backfilled.to_csv('/home/cs231n/data/selected_snapshots_full_deduped_backfilled.csv')

In [38]:
to_whiten = [
 'Ambient Air Temp',
 'Barometric Press',
 'Bus Utilization',
 'Cat Intake Gas Temp',
 'Cat Outlet Gas Temp',
 'Cmd Eng Fuel Press',
 'Drvr Demand Torque',
 'Eng Air Flow Rate',
 'Eng Avg Fuel Econ',
 'Eng Coolant Temp',
 'Eng Demand Torque',
 'Eng DPF Intake Press',
 'Eng Egr Valve Pos',
 'Eng Exhaust Gas Temp',
 'Eng Fuel Del Press',
 'EngFuelTemp1',
 'Engine Speed',
 'Eng Man Abs Pressure',
 'Eng Oil Pressure',
 'EngInjRail1Press',
 'EngIntakeMan1Temp',
 'EngOilTemp1',
 'Eng Percent Torque',
 'EngTurbo1Boost',
 'EngTurbo1Pos',
 'EngTurbo1Speed',
 'Event - All Lamps On Time Hr',
 'Event - Amber Lamp Time Hr',
 'Event - Mil Lamp Time Hr',
 'Event - Red Lamp Time Hr',
 'Exhaust Tank Level',
 'Exhaust Tank Temp',
 'Fan Speed',
 'Keyswitch Bat Pot',
 'Part Trap Diff Press',
 'Part Trap Out Temp',
 'Scr Intake Gas Temp',
 'Scr Outlet Gas Temp',
 'Vehicle Speed',
 'DTCID',
 'Trip Distance',
 'Trip Idle Time',
 'Trip Run Time',
 'Altitude',
 'Engine Start Ambient',
 'Engine Start Coolant',
 'Latitude',
 'Longitude',
 'Lifetime Idle Hours',
 'Lifetime Idle Fuel',
 'Lifetime Fuel',
 'Lifetime Distance']

In [40]:
selected_snapshots_full_deduped_backfilled_whitened = selected_snapshots_full_deduped_backfilled.drop(['Population'],1)

In [42]:
for col in to_whiten:
    selected_snapshots_full_deduped_backfilled_whitened[col] = preprocessing.scale(selected_snapshots_full_deduped_backfilled_whitened[col])

/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [44]:
selected_snapshots_full_deduped_backfilled_whitened.to_csv('/home/cs231n/data/selected_snapshots_full_deduped_backfilled_whitened.csv')

In [46]:
selected_snapshots_full_deduped_backfilled_whitened.to_pickle('/home/cs231n/data/selected_snapshots_full_deduped_backfilled_whitened.pkl')

MemoryError: 

In [ ]:
category_var = ['Veh Ref ID','Event Type Description','Brake Switch','Clutch Switch','Cruise Status','Dpf Regen Inhibit Sw', 
                'Dpf Thermal Mngmnt','Eng Coolant Level','Population','DTCID','state']

for i in category_var:
    selected_snapshots[i] = selected_snapshots[i].astype('category')

In [10]:
selected_snapshots = selected_snapshots.fillna(method='bfill')
selected_repairs = selected_repairs.fillna(method='bfill')

In [11]:
selected_snapshots.to_csv('/home/cs231n/data/snapshots_full_cleaned.csv')
selected_repairs.to_csv('/home/cs231n/data/repairs_full_cleaned.csv')

In [6]:
# 6/14/2016 20:17:07
s_to_date = lambda x: pandas.to_datetime(x, format='%m/%d/%Y %H:%M:%S')
snapshots_df = pandas.read_csv('/home/cs231n/data/snapshots_full_cleaned.csv', converters={'Event DateTime':s_to_date})

In [14]:
# 2/3/2016
r_to_date = lambda x: pandas.to_datetime(x, format='%m/%d/%Y')
repairs_df = pandas.read_csv('/home/cs231n/data/repairs_full_cleaned.csv', converters={'Rpr_Dt':r_to_date})

In [7]:
snapshots_deduped = snapshots_df.drop_duplicates()

In [8]:
len(snapshots_df)

1324311

In [9]:
len(snapshots_deduped)

1324311

In [ ]:
category_var = ['Veh Ref ID','Event Type Description','Brake Switch','Clutch Switch','Cruise Status','Dpf Regen Inhibit Sw', 
                'Dpf Thermal Mngmnt','Eng Coolant Level','Population','DTCID','state']

for i in category_var:
    selected_snapshots[i] = selected_snapshots[i].astype('category')

In [28]:
snapshots_df = snapshots_df.drop(['Unnamed: 0'],1)

In [30]:
repairs_df = repairs_df.drop(['Unnamed: 0'],1)

In [31]:
snapshots_df.to_csv('/home/cs231n/data/snapshots_cleaned_formatted.csv')
repairs_df.to_csv('/home/cs231n/data/repairs_cleaned_formatted.csv')

In [5]:
snapshots_df.to_csv('/home/cs231n/data/snapshots_cleaned_formatted.csv')

In [7]:
s_veh_key = 'Veh Ref ID'
r_veh_key = 'Chassis Reference Number'
s_time_key = 'Event DateTime'
r_time_key = 'Rpr_Dt'

In [8]:
def get_slices(k, spacing, num_slices, snapshots):
    slices = []
    for i in range(k):
        start = num_slices - (i+1)*spacing
        end = num_slices - i*spacing
        
        if i == k-1 and num_slices - start > 0: # last iter
            start = 0
            
        if start >=0 and end >=0:
            slices.append((i, snapshots[start : end]))
        else:
            break
            
    return slices

In [68]:
def get_repair_slices_map(veh_ids, snapshots, repairs, k=10, spacing=10, code='ATA9'):
    repair_slices = {}
    for veh_id in veh_ids:
        v_snapshots = snapshots[snapshots[s_veh_key] == veh_id].sort_values(by=s_time_key)
        v_repairs = repairs[repairs[r_veh_key] == veh_id].sort_values(by=r_time_key)

        start_date = pandas.to_datetime('1/1/2000') ## in past so first snapshot is captured

        grouped_repairs = v_repairs.groupby([code])
        
        if len(grouped_repairs) == 0:
            continue

        ## Best indicator of repair type is the ATA9 code
        ## Iterate over each repair type and append slices
        for repair_type, repair_group in grouped_repairs:
            start = start_date

            ## null check
            if repair_type not in repair_slices:
                repair_slices[repair_type] = {}
            veh_slices_repair = repair_slices[repair_type]
            
            ## for each repair type, grab slices of snapshots
            for end in repair_group[r_time_key]:
                range_mask = (v_snapshots[s_time_key] >= start) & (v_snapshots[s_time_key] <= end)
                num_slices = len(v_snapshots[range_mask])
                
                for i,slices in get_slices(k, spacing, num_slices, v_snapshots):
                    if len(slices) > 0:
                        if i not in veh_slices_repair:
                            veh_slices_repair[i] = []
                        veh_slices_repair[i].append(slices)

                ## reset start to end for next iteration
                start = end

    return repair_slices

In [121]:
ata6_train_repairs['Chassis Reference Number'] = ata6_train_repairs['Chassis\r\nReference\r\nNumber']
ata6_val_repairs['Chassis Reference Number'] = ata6_val_repairs['Chassis\r\nReference\r\nNumber']
ata6_test_repairs['Chassis Reference Number'] = ata6_test_repairs['Chassis\r\nReference\r\nNumber']

In [11]:
ata6_train_repairs = pandas.read_csv('ATA6_train.csv')
ata6_val_repairs = pandas.read_csv('ATA6_val.csv')
ata6_test_repairs = pandas.read_csv('ATA6_test.csv')

In [15]:
k = 10
spacing = 10
code = 'ATA6'

In [2]:
val_veh_ids = ata6_val_repairs['Chassis Reference Number'].unique()
train_veh_ids = ata6_train_repairs['Chassis Reference Number'].unique()
test_veh_ids = ata6_test_repairs['Chassis Reference Number'].unique()

NameError: name 'ata6_val_repairs' is not defined

In [1]:
val_veh_ids

NameError: name 'val_veh_ids' is not defined

In [69]:
val_repairs_map = get_repair_slices_map(val_veh_ids, snapshots_df, ata6_val_repairs, k=k, spacing=spacing, code=code)
train_repairs_map = get_repair_slices_map(train_veh_ids, snapshots_df, ata6_train_repairs, k=k, spacing=spacing, code=code)
test_repairs_map = get_repair_slices_map(test_veh_ids, snapshots_df, ata6_test_repairs, k=k, spacing=spacing, code=code)

283

In [81]:
def save_map(repairs_map, filename):
    for code in repairs_map.keys():
        for window in repairs_map[code].keys():  
            for i in range(len(repairs_map[code][window])):
                n = "{}_{}_{}_{}_.pkl".format(filename, code, window, i)
                repairs_map[code][window][i].to_pickle(n)

def open_map(filename):
    repairs_map = {}
    for file in glob.glob("{}*".format(filename)):
        keys = file.split("_")
        if len(keys) == 5:
            code = int(keys[1])
            window = int(keys[2])
            i = int(keys[3])
            n = "{}_{}_{}_{}_.pkl".format(filename, code, window, i)            
            repairs_map_slice = pandas.read_pickle(n)
            if code not in repairs_map:
                repairs_map[code] = {}
            if window not in repairs_map[code]:
                repairs_map[code][window] = []
            repairs_map[code][window].append(repairs_map_slice)

    return repairs_map

In [75]:
save_map(val_repairs_map, 'val')
save_map(train_repairs_map, 'train')
save_map(test_repairs_map, 'test')

KeyError: 0

In [82]:
save_map(val_repairs_map, '/home/cs231n/data/val')

In [83]:
val_map_revived = open_map('/home/cs231n/data/val')

In [87]:
len(val_repairs_map[44004][4][0])

10

In [88]:
val_map_revived[44003][4][0]

,Unnamed: 0,Veh Ref ID,Event DateTime,Event Type Description,Acc Pedal Position,Ambient Air Temp,Barometric Press,Brake Switch,Bus Utilization,Cat Intake Gas Temp,...,Altitude,Engine Start Ambient,Engine Start Coolant,Latitude,Longitude,Lifetime Idle Hours,Lifetime Idle Fuel,Lifetime Fuel,Lifetime Distance,Lifetime Engine Hours
525997,630394,59.0,2016-06-19 01:05:04,trip_periodic,96.8,29.00,93.0,0.0,58.0,322.09,...,2538.0,36.66,83.88,40.86,-100.03,210.60,140.15,6754.67,48655.87,1065.15
525998,630395,59.0,2016-06-19 01:05:04,trip_periodic,96.8,29.00,93.0,0.0,58.0,322.09,...,2538.0,36.66,83.88,40.86,-100.03,210.60,140.15,6754.67,48655.87,1065.15
526036,630433,59.0,2016-06-19 03:05:04,trip_periodic,0.0,27.18,91.5,0.0,53.0,118.87,...,2971.0,36.66,83.88,41.13,-101.12,211.65,140.81,6763.17,48716.77,1067.15
526035,630432,59.0,2016-06-19 03:05:04,trip_periodic,0.0,27.18,91.5,0.0,53.0,118.87,...,2971.0,36.66,83.88,41.13,-101.12,211.65,140.81,6763.17,48716.77,1067.15
526033,630430,59.0,2016-06-19 03:05:24,trip_end,0.0,27.28,91.5,0.0,53.0,118.59,...,2974.0,36.66,83.88,41.13,-101.12,211.65,140.81,6763.17,48716.77,1067.15
526034,630431,59.0,2016-06-19 03:05:24,trip_end,0.0,27.28,91.5,0.0,53.0,118.59,...,2974.0,36.66,83.88,41.13,-101.12,211.65,140.81,6763.17,48716.77,1067.15
526041,630438,59.0,2016-06-19 13:02:36,trip_start,0.0,25.78,91.0,0.0,52.0,53.00,...,2981.0,24.99,33.88,41.13,-101.12,211.65,140.81,6763.17,48716.77,1067.15
526042,630439,59.0,2016-06-19 13:02:36,trip_start,0.0,25.78,91.0,0.0,52.0,53.00,...,2981.0,24.99,33.88,41.13,-101.12,211.65,140.81,6763.17,48716.77,1067.15
525976,630373,59.0,2016-06-19 16:02:36,trip_periodic,70.4,29.18,84.5,0.0,52.0,337.00,...,5221.0,24.99,33.88,41.14,-104.17,212.10,141.21,6788.42,48880.77,1070.15
525975,630372,59.0,2016-06-19 16:02:36,trip_periodic,70.4,29.18,84.5,0.0,52.0,337.00,...,5221.0,24.99,33.88,41.14,-104.17,212.10,141.21,6788.42,48880.77,1070.15
